In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload conll_raw_sample.txt

In [ ]:
def read_conll(file_path):
    sentences = []
    tokens, labels = [], []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    sentences.append((tokens, labels))
                    tokens, labels = [], []
            else:
                splits = line.split()
                if len(splits) >= 2:
                    tokens.append(splits[0])
                    labels.append(splits[1])
    if tokens:
        sentences.append((tokens, labels))
    return sentences

data = read_conll("data/conll_raw_sample.txt")


In [ ]:
from datasets import Dataset

tokens = [x[0] for x in data]
ner_tags = [x[1] for x in data]

label_list = sorted(set(tag for seq in ner_tags for tag in seq))
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

tag_ids = [[label2id[tag] for tag in seq] for seq in ner_tags]

dataset = Dataset.from_dict({"tokens": tokens, "ner_tags": tag_ids})
dataset = dataset.train_test_split(test_size=0.2)


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "FacebookAI/xlm-roberta-base"  # Or use your preferred model
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
)


In [ ]:
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(
        example["tokens"],
        truncation=True,
        padding='max_length',  # ✅ Add this
        is_split_into_words=True,
        max_length=128         # ✅ Optionally limit sequence length
    )

    labels = []
    word_ids = tokenized_inputs.word_ids()
    prev_word_id = None

    for word_id in word_ids:
        if word_id is None:
            labels.append(-100)
        elif word_id != prev_word_id:
            labels.append(example["ner_tags"][word_id] if word_id < len(example["ner_tags"]) else -100)
        else:
            labels.append(example["ner_tags"][word_id] if word_id < len(example["ner_tags"]) else -100)
        prev_word_id = word_id

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=False)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./amharic-ner-results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)


In [ ]:
from seqeval.metrics import classification_report
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[pred] for pred, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return classification_report(true_labels, true_predictions, output_dict=True)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


In [ ]:
results = trainer.evaluate()
print("Validation Results:", results)

# Save model and tokenizer locally
model.save_pretrained("./amharic-ner-model")
tokenizer.save_pretrained("./amharic-ner-model")

In [ ]:
# Import libraries
import shap
import lime
import lime.lime_text
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from datasets import Dataset
import pandas as pd
from seqeval.metrics import classification_report
import os
from google.colab import files

# Define paths
model_path = "./amharic-ner-model"
output_report = "./amharic_ner_interpretability_report.md"

# Load the fine-tuned model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)
id2label = model.config.id2label
label2id = model.config.label2id

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Recreate test dataset
def read_conll(file_path):
    sentences = []
    tokens, labels = [], []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    sentences.append((tokens, labels))
                    tokens, labels = [], []
            else:
                splits = line.split()
                if len(splits) >= 2:
                    tokens.append(splits[0])
                    labels.append(splits[1])
    if tokens:
        sentences.append((tokens, labels))
    return sentences

# Check for data file
if not os.path.exists("conll_raw_sample.txt"):
    print("Please upload conll_raw_sample.txt")
    uploaded = files.upload()
    if not os.path.exists("conll_raw_sample.txt"):
        raise FileNotFoundError("conll_raw_sample.txt not found.")

data = read_conll("conll_raw_sample.txt")
tokens = [x[0] for x in data]
ner_tags = [x[1] for x in data]
label_list = sorted(set(tag for seq in ner_tags for tag in seq))  # Fixed NameError
tag_ids = [[label2id[tag] for tag in seq] for seq in ner_tags]
dataset = Dataset.from_dict({"tokens": tokens, "ner_tags": tag_ids})
dataset = dataset.train_test_split(test_size=0.2)
test_dataset = dataset["test"]

# Tokenize test dataset
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(
        example["tokens"],
        truncation=True,
        padding='max_length',
        is_split_into_words=True,
        max_length=128
    )
    labels = []
    word_ids = tokenized_inputs.word_ids()
    prev_word_id = None
    for word_id in word_ids:
        if word_id is None:
            labels.append(-100)
        elif word_id != prev_word_id:
            labels.append(example["ner_tags"][word_id] if word_id < len(example["ner_tags"]) else -100)
        else:
            labels.append(example["ner_tags"][word_id] if word_id < len(example["ner_tags"]) else -100)
        prev_word_id = word_id
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_test_dataset = test_dataset.map(tokenize_and_align_labels, batched=False)

# Initialize NER pipeline
ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

# Function to prepare input for SHAP
def predict_proba(texts):
    # Handle 2D input by flattening if necessary
    if isinstance(texts, np.ndarray) and texts.ndim > 1:
        texts = texts.flatten()
    inputs = tokenizer(texts.tolist() if isinstance(texts, np.ndarray) else texts,
                      return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs).logits
        probs = torch.softmax(outputs, dim=-1).cpu().numpy()
    return probs.reshape(-1, len(id2label))

# SHAP Explainer with corrected reference data
reference_data = np.array(["placeholder"]).reshape(1, -1)  # Fixed for IndexError
explainer_shap = shap.KernelExplainer(predict_proba, reference_data)

# Function for LIME explanation
def lime_predict_proba(texts):
    return predict_proba(texts)

# LIME Explainer
lime_explanation = lime.lime_text.LimeTextExplainer(class_names=list(label2id.keys()), bow=False)

# Analyze a sample from the test set
sample = test_dataset[0]
tokens = sample["tokens"]
true_labels = [id2label[l] for l in sample["ner_tags"] if l != -100]
text_input = " ".join(tokens)

# Get SHAP values
try:
    shap_values = explainer_shap.shap_values(text_input, nsamples=50)
except Exception as e:
    print(f"SHAP error: {e}. Reducing nsamples or skipping SHAP.")
    shap_values = "SHAP computation failed."

# Get LIME explanation
try:
    lime_explanation = lime_explainer.explain_instance(text_input, lime_predict_proba, num_features=10, labels=range(len(id2label)))
    lime_explanation_list = lime_explanation.as_list()
except Exception as e:
    print(f"LIME error: {e}. Skipping LIME.")
    lime_explanation_list = "LIME computation failed."

# Predict with pipeline
predictions = ner_pipeline(text_input)
pred_labels = [pred["entity"] for pred in predictions if "entity" in pred]

# Analyze difficult cases
difficult_cases = []
for idx, example in enumerate(test_dataset):
    tokens = example["tokens"]
    true_tags = [id2label[l] for l in example["ner_tags"] if l != -100]
    text = " ".join(tokens)
    preds = ner_pipeline(text)
    pred_tags = [pred["entity"] for pred in preds if "entity" in pred]
    min_len = min(len(true_tags), len(pred_tags))
    true_tags = true_tags[:min_len]
    pred_tags = pred_tags[:min_len]
    if min_len > 0:
        incorrect = [t != p for t, p in zip(true_tags, pred_tags)]
        if any(incorrect):
            difficult_cases.append({
                "index": idx,
                "tokens": tokens,
                "true_labels": true_tags,
                "pred_labels": pred_tags,
                "incorrect_positions": [i for i, x in enumerate(incorrect) if x]
            })

# Generate classification report
true_all = []
pred_all = []
for example in test_dataset:
    true_tags = [id2label[l] for l in example["ner_tags"] if l != -100]
    text = " ".join(example["tokens"])
    preds = ner_pipeline(text)
    pred_tags = [pred["entity"] for pred in preds if "entity" in pred]
    min_len = min(len(true_tags), len(pred_tags))
    true_all.append(true_tags[:min_len])
    pred_all.append(pred_tags[:min_len])
report_dict = classification_report(true_all, pred_all, output_dict=True)

# Generate Interpretability Report
report = f"""
# Amharic NER Model Interpretability Report

## 1. Model Overview
- **Model**: {model_path} (based on xlm-roberta-base)
- **Task**: Named Entity Recognition (NER) for Amharic text
- **Labels**: {list(id2label.values())}

## 2. SHAP Analysis
### Sample Text: {" ".join(sample["tokens"])}
### SHAP Insights:
- Top contributing tokens for each label:
{shap_values}

## 3. LIME Analysis
### LIME Explanation for Sample:
{lime_explanation_list}

## 4. Performance Metrics
### Classification Report:
{pd.DataFrame(report_dict).T.to_markdown()}

## 5. Difficult Cases Analysis
### Number of Difficult Cases: {len(difficult_cases)}
### Example Difficult Case:
{difficult_cases[0] if difficult_cases else "No difficult cases found."}

## 6. Recommendations for Improvement
- **Ambiguous Entities**: If 'LOC' performance is low, collect more diverse location data.
- **Overlapping Entities**: Enhance tokenization to handle multi-word entities.
- **Data Augmentation**: Use synthetic Amharic NER data to increase dataset size.
- **Hyperparameter Tuning**: Try learning rates (e.g., 5e-5) or more epochs (e.g., 5).
"""

# Save report
with open(output_report, "w", encoding="utf-8") as f:
    f.write(report)

# Download report
files.download(output_report)

print("Interpretability analysis completed. Report saved and downloaded as 'amharic_ner_interpretability_report.md'.")